In [66]:
from pyspark.sql import SparkSession
from pyspark.sql.types import IntegerType, FloatType, StringType
spark = SparkSession.builder.\
    config('spark.jars.packages','org.mongodb.spark:mongo-spark-connector_2.11:2.3.2').getOrCreate()
lenta = spark.read.format('com.mongodb.spark.sql.DefaultSource').option("uri", "mongodb://127.0.0.1/food_db.lenta").load()
metro = spark.read.format('com.mongodb.spark.sql.DefaultSource').option("uri", "mongodb://127.0.0.1/food_db.metro").load()
prisma = spark.read.format('com.mongodb.spark.sql.DefaultSource').option("uri", "mongodb://127.0.0.1/food_db.prisma").load()
carusel = spark.read.format('com.mongodb.spark.sql.DefaultSource').option("uri", "mongodb://127.0.0.1/food_db.carusel").load()

lenta = lenta.withColumnRenamed("data-old-price","old_price")
metro = metro.withColumnRenamed("data-old-price","old_price")
prisma = prisma.withColumnRenamed("data-old-price","old_price")
carusel = carusel.withColumnRenamed("data-old-price","old_price")
l = lenta.alias('l')
m = metro.alias('m')
p = prisma.alias('p')
c = carusel.alias('c')
l.printSchema()
m.printSchema()
p.printSchema()
c.printSchema()

root
 |-- _id: struct (nullable = true)
 |    |-- oid: string (nullable = true)
 |-- amount: string (nullable = true)
 |-- data-energy-drink: string (nullable = true)
 |-- data-group-quantity: string (nullable = true)
 |-- old_price: string (nullable = true)
 |-- data-old-price-per-kg: string (nullable = true)
 |-- datetm: timestamp (nullable = true)
 |-- item_weight: string (nullable = true)
 |-- name: string (nullable = true)
 |-- price: string (nullable = true)
 |-- price_per_kg: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- type: string (nullable = true)
 |-- url: string (nullable = true)
 |-- weight: string (nullable = true)

root
 |-- _id: struct (nullable = true)
 |    |-- oid: string (nullable = true)
 |-- amount: string (nullable = true)
 |-- data-energy-drink: string (nullable = true)
 |-- data-group-quantity: string (nullable = true)
 |-- old_price: string (nullable = true)
 |-- data-old-price-per-kg: string (nullable = true)
 |-- datetm: timestamp 

In [67]:
joined = l.join(m, ['name', 'weight']). \
    join(p, ['name', 'weight']).\
    join(c, ['name', 'weight']).\
    selectExpr('l.name as l_name', 'l.weight as l_weight',
               'l.price as l_price', 'm.price as m_price', 'p.price as p_price', 'c.price as c_price',
               'l.old_price as l_old_price', 'm.old_price as m_old_price', 'p.old_price as p_old_price', 'c.old_price as c_old_price',
               ).sort('name', 'weight')

In [68]:
print('before distinct', joined.count())
joined = joined.distinct()
print('after distinct', joined.count())

before distinct 31710
after distinct 1723


In [69]:
print(joined.count())
joined.show(truncate=50)

1723
+--------------------------------------------------+--------+-------+-------+-------+-------+-----------+-----------+-----------+-----------+
|                                            l_name|l_weight|l_price|m_price|p_price|c_price|l_old_price|m_old_price|p_old_price|c_old_price|
+--------------------------------------------------+--------+-------+-------+-------+-------+-----------+-----------+-----------+-----------+
|        Драже Клубничный микс ТМ Tic Tac (Тик Так)|      16|  42.19|   47.0|   44.9|   47.0|       null|       null|       null|       null|
|Лапша быстрого приготовления вкус курицы ТМ Дош...|      70|  19.89|  16.48|   18.9|  16.48|       null|       null|       null|       null|
|Средство для мытья посуды Нежные ручки Ромашка ...|     900| 177.59|   99.0|  139.9|   99.0|       null|      169.0|      199.9|      169.0|
|               Форель слабосоленая ТМ Русское Море|     120| 283.09|  310.0|  346.9|  310.0|       null|       null|       null|       null|
|

In [70]:
joined.coalesce(1).write.csv('joined.csv', mode='overwrite', sep='\t', nullValue = '\u0000',
    emptyValue = '\u0000', header=True)

In [84]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import regexp_replace
spark = SparkSession.builder.\
    config('spark.jars.packages','org.mongodb.spark:mongo-spark-connector_2.11:2.3.2').getOrCreate()
c = spark.read.json('/home/comp/Documents/src/food/data_20_03_15-16_35_58_.json')
c.printSchema()
print(c)
c.show()
c = c.drop('_id', 'insert_datetm')
c.write.parquet('./store/carusel.parquet')
c.write.csv('./store/carusel.csv')
c.write.json('./store/caruse.json')

root
 |-- cat: string (nullable = true)
 |-- data-amount: string (nullable = true)
 |-- data-energy-drink: string (nullable = true)
 |-- data-group-quantity: string (nullable = true)
 |-- data-item-weight: string (nullable = true)
 |-- data-old-price: string (nullable = true)
 |-- data-old-price-per-kg: string (nullable = true)
 |-- data-price: string (nullable = true)
 |-- data-price-per-kg: string (nullable = true)
 |-- data-product-id: string (nullable = true)
 |-- data-type: string (nullable = true)
 |-- data-weight: string (nullable = true)
 |-- datetm: string (nullable = true)
 |-- name: string (nullable = true)
 |-- shop: string (nullable = true)
 |-- url: string (nullable = true)

DataFrame[cat: string, data-amount: string, data-energy-drink: string, data-group-quantity: string, data-item-weight: string, data-old-price: string, data-old-price-per-kg: string, data-price: string, data-price-per-kg: string, data-product-id: string, data-type: string, data-weight: string, datetm: s

In [53]:
from pyspark.sql.types import IntegerType, FloatType, StringType
from pyspark.sql.functions import regexp_replace, udf, when, round

from pyspark.sql import SparkSession
spark = SparkSession.builder.\
    config('spark.jars.packages','org.mongodb.spark:mongo-spark-connector_2.11:2.3.2').getOrCreate()
c = spark.read.json('/home/comp/Documents/src/food/data_20_03_15-16_35_58_.json')
c.printSchema()
print(c)
c.show()

root
 |-- cat: string (nullable = true)
 |-- data-amount: string (nullable = true)
 |-- data-energy-drink: string (nullable = true)
 |-- data-group-quantity: string (nullable = true)
 |-- data-item-weight: string (nullable = true)
 |-- data-old-price: string (nullable = true)
 |-- data-old-price-per-kg: string (nullable = true)
 |-- data-price: string (nullable = true)
 |-- data-price-per-kg: string (nullable = true)
 |-- data-product-id: string (nullable = true)
 |-- data-type: string (nullable = true)
 |-- data-weight: string (nullable = true)
 |-- datetm: string (nullable = true)
 |-- name: string (nullable = true)
 |-- shop: string (nullable = true)
 |-- url: string (nullable = true)

DataFrame[cat: string, data-amount: string, data-energy-drink: string, data-group-quantity: string, data-item-weight: string, data-old-price: string, data-old-price-per-kg: string, data-price: string, data-price-per-kg: string, data-product-id: string, data-type: string, data-weight: string, datetm: s

In [54]:
c = c.drop('data-group-quantity', 'data-energy-drink', 'data-amount', 'data-item-weight', 'data-energy-drink'). \
    withColumn('old_price', c['data-old-price'].cast(FloatType())). \
    withColumn('old_price_per_kg', c['data-old-price-per-kg'].cast(FloatType())). \
    withColumn('price', c['data-price'].cast(FloatType())). \
    withColumn('price_per_kg', c['data-price-per-kg'].cast(FloatType())). \
    withColumn('product_id', c['data-product-id'].cast(IntegerType())). \
    withColumn('type', c['data-type'].cast(StringType())). \
    withColumn('weight', c['data-weight'].cast(IntegerType()))

calc_discount = round((1 - c.price/c.old_price) * 100, 2)

c = c.withColumn('discount', when(c.old_price.isNull(), None).otherwise(calc_discount)). \
    select('shop', 'name', 'weight', 'price', 'old_price', 'price_per_kg',
         'old_price_per_kg', 'discount', 'cat', 'product_id', 'type', 'datetm', 'url')
c.show()

+-----+--------------------+------+------+---------+------------+----------------+--------+---+----------+-----+--------------------+--------------------+
| shop|                name|weight| price|old_price|price_per_kg|old_price_per_kg|discount|cat|product_id| type|              datetm|                 url|
+-----+--------------------+------+------+---------+------------+----------------+--------+---+----------+-----+--------------------+--------------------+
|metro|Микро зелень Керв...|    10| 119.9|     null|     11990.0|            null|    null|  1|   4047135|piece|2020-03-15 16:36:...|/products/138039-...|
|metro|Кешью обжаренный ...|    50|499.89|     null|      9997.8|            null|    null|  1|    137980|piece|2020-03-15 16:36:...|/products/31423-k...|
|metro|Фундук обжаренный...|   150|469.89|     null|      3132.6|            null|    null|  1|    669869|piece|2020-03-15 16:36:...|/products/61493-f...|
|metro|Смесь соленая с о...|   800| 839.9|     null|     1049.88|     

In [55]:
c.sort('discount', ascending=False).show()

+-----+--------------------+------+-----+---------+------------+----------------+--------+---+----------+-----+--------------------+--------------------+
| shop|                name|weight|price|old_price|price_per_kg|old_price_per_kg|discount|cat|product_id| type|              datetm|                 url|
+-----+--------------------+------+-----+---------+------------+----------------+--------+---+----------+-----+--------------------+--------------------+
|metro|Печенье бисквитно...|   270| 40.0|    101.0|      148.15|       374.07407|    60.4|  9|    136432|piece|2020-03-15 17:09:...|/products/11848-p...|
|metro|Печенье бисквитно...|   270| 40.0|    101.0|      148.15|       374.07407|    60.4|  9|    138455|piece|2020-03-15 17:07:...|/products/11849-p...|
|metro|Печенье бисквитно...|   270| 40.0|    101.0|      148.15|       374.07407|    60.4| 83|    138455|piece|2020-03-15 17:10:...|/products/11849-p...|
|metro|Печенье бисквитно...|   270| 40.0|    101.0|      148.15|       374.0

In [5]:
print(1)

1


In [90]:
c.write.mode('append').partitionBy('shop', 'cat').parquet('./store/data.parquet')